<a href="https://colab.research.google.com/github/reisdout/FederatedColab/blob/main/federated_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import time
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import RegressorMixin
from tensorflow.python import training

https://ai-pool.com/d/how-to-get-the-weights-of-keras-model-

Keras has implemented some functions for getting or setting weights for every layer. 

    layer.get_weights(): returns the weights of the layer as a list of Numpy arrays.
    layer.set_weights(weights): sets the weights of the layer from a list of Numpy arrays.

Using these functions you can write a piece of code to get all layers' weights

for layer in model.layers:
    weights = layer.get_weights() # list of numpy arrays

Or you can get the weights right from the model

from keras.models import Sequential

model = Sequential()
# ...
weights = model.get_weights() # returs a numpy list of weights

Keras model also has get_weights() and set_weights(weights) functions like every layer has.

If you need more take a look at this keras doc.






https://stackoverflow.com/questions/47183159/how-to-set-weights-in-keras-with-a-numpy-array

The set_weights() method of keras accepts a list of numpy arrays, what you have passed to the method seems like a single array. The shape of this should be the same as the shape of the output of get_weights() on the same layer. Here's the code:
```
l=[]
x=np.array() #weights
y=np.array() #array of biases
l.append(x)
l.append(y)
```

loaded_model.layers[0].set_weights(l) #loaded_model.layer[0] being the layer


In [14]:
'''
  
class Server_FederatedOMS:
  
  In the central server, we decide to use only the highest
  accuracy holder model as the central server updated model, send it to the local
  clients after the computation.



  def ReceiveModelsFromClients(self, parIdCliente):
    print("Recebido Modelo do Cliente 1")
  def Consolidar(self):
    print("Consolidado todos os modelos")
  def FeedBackConsolidatedModel(self):
    print ("Modelos enviados")

  clients = ['Cliente1', 'Cliente2']


class Server_FederatedBMA():

owever, in the BMA technique, the central server receives
four models with model accuracy performances from the local servers or clients.
In the central server, we sort the model using their performances. Then we
decide to use the two best models or half of the models based on performances.
Then BMA technique loops through each model’s hidden layers and neurons to do
the sum of the weights and average them accordingly.

  def ReceiveModelsFromClients(self)
  def Consolidar(self)
  def FeedBackConsolidatedModel(self)

    clients[] = ['Cliente1', 'Cliente2']



    def __init__(self, parCentralServer, parId):
      self.id=parId
      self.centralServer = parCentralServer
      #self.centralServer.RegisterClient(self,self)


'''
class Client():



    #resultadoTreinamento = np.eye(10)

    def __init__(self,parId,parTraininPath, parTestPath, parPrevisionWindow):
      self.id=parId
      self.trainingPath=parTraininPath
      self.testPath = parTestPath
      self.T = parPrevisionWindow
      #centralServer = Server_FederatedAMA()
      #confusionMatrizModelClient = np.full((2,2), 1)
      #confusionMatrizModelServer = np.full((2,2), 2)
      self.currentConfusionMatriz =np.full((2,2), 0)
      self.weightsClientModel = np.array([])
      self.weightsServerModel = []
      #self.base = pd.DataFrame()
      #self.base_treinamento =  np.array([])
      self.real_congestion_test = np.array([])
      #self.test_vectors = []
      #self.previsores = []
      #self.real_congestion = []
      #self.regressor = Sequential()
      self.input_shape =0;
      self.len_base_teste = 0;
      #self.centralServer.RegisterClient(self,self)

    def EvaluateServerModel(self):
      print ("Modelo do Servidor avaliado")

    def LoadTainingDataSet(self):
      base = pd.read_csv(self.trainingPath)
      base = base.dropna()
      base_treinamento = base.iloc[:, [1,2,3,5]].values
      previsores=[]
      real_congestion = []
      for i in range(self.T, base_treinamento.shape[0]): #base_treinamento.shape[0] número de linhas dos dados de treinamento
        previsores.append(base_treinamento[i-self.T:i, 0:4])
        real_congestion.append(base_treinamento[i, 3])#O resultado é do último cara
      previsores, real_congestion = np.array(previsores), np.array(real_congestion)
      self.input_shape = previsores.shape[1]
      return previsores, real_congestion


    def RefreshModel(self, parInitial=False): #Constroi na primeira vez e atualiza, a partir da avaliação do servidor cetral
      #pensar melhor no critério
      previsores,real_congestion = self.LoadTainingDataSet()
      regressor = Sequential()
      
      if(parInitial):#tem que construir a rede do zero e treinar os pesos
        regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (self.input_shape, 4)))# 4, pois são 4 previsores
        regressor.add(Dropout(0.3)) #zerar 30% das entradas para evitar o overfiting
        regressor.add(LSTM(units = 50, return_sequences = True))
        regssor.add(Dropout(0.3))
        regressor.add(LSTM(units = 50, return_sequences = True))
        regressor.add(Dropout(0.3))
        regressor.add(LSTM(units = 50))
        regressor.add(Dropout(0.3))

        '''
        Segundo https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
        as saídas de cada unidade da LSTM e, portanto, a saída global é a dimenção do número de previsores, que, no nosso
        caso, é 4. Daí esses 4 estão sendo levados em um softmax de tres neurônios, pois há tres categorias no final"
        ''' 
        regressor.add(Dense(units = 1, activation = 'sigmoid', name="client_"+str(self.id)))
        regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
        es = EarlyStopping(monitor = 'loss', min_delta = 1e-10, patience = 10, verbose = 1)
        rlr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 5, verbose = 1)
        mcp = ModelCheckpoint(filepath = 'pesos.h5', monitor = 'loss', 
                      save_best_only = True, verbose = 1)
        regressor.fit(previsores, real_congestion, epochs = 100, batch_size = 32,
                      callbacks = [es, rlr, mcp])
        self.weightsClientModel = regressor.get_weights()
      else:
        if(self.ServerModelIsBetter()):
          print("Pesos atualizados de acordo com o modelo do servidor")
        else:
           print("Pesos Mantidos de acordo com o modelo do cliente")

    def GetMapedMatrix(self,parPrevisoes):
      classe_teste2 = np.array([])
      previsoes2 =  np.array([])

      for i in range(0, self.len_base_teste):
        if(self.real_congestion_test[i] < 0.3):
          classe_teste2 = np.append(classe_teste2,0)
        elif (self.real_congestion_test[i,0] >= 0.3 and self.real_congestion_test[i] < 0.75):
          classe_teste2= np.append(classe_teste2,1)
        else:
          classe_teste2 = np.append(classe_teste2,2)

      for i in range(0, len(parPrevisoes)):
        if(parPrevisoes[i] < 0.3):
          previsoes2= np.append(previsoes2,0)
        elif (parPrevisoes[i] >= 0.3 and parPrevisoes[i] < 0.75):
          previsoes2= np.append(previsoes2,1)
        else:
          previsoes2 = np.append(previsoes2,2)

      return classe_teste2, previsoes2


    def EvalueteServerModel(self, parServerModel):

      test_vectors = self.LoadTestData()
      previsoes = parServerModel.predict(test_vectors)
      updated = False
      '''
      Observe que os previsores teste tem 90 quádruplas que conduzem ao resultado
      do último estado, da quádrupla 90. Prontamente preparado para pevisões....
      '''
      classe_teste2,previsoes2 = self.GetMapedMatrix(previsoes)
      matriz = confusion_matrix(classe_teste2,previsoes2)
      currentSum = 0
      newSum = 0
      if(self.currentConfusionMatriz.ndim > 1): # as vezes a rede pode errar ao ponto de dar só uma categoria, daí cai no else...
        for i in range (0, len(self.currentConfusionMatriz)):
            currentSum = currentSum + self.currentConfusionMatriz[i][i]
      else:
        currentSum = self.currentConfusionMatriz[i]
      if(matriz.ndim > 1):
        for i in range (0, len(matriz)):
          newSum = newSum + matriz[i][i]
      else:
        newSum = matriz[i]
      if(newSum > currentSum):
        self.currentConfusionMatriz = np.array(matriz)
        self.regressor.set_weights(self.weightsServerModel)
        updated = True
      print ("Modelo do Servidor avaliado")
      return updated 

     


    def LoadTestData(self):
      base = pd.read_csv(self.trainingPath)
      base = base.dropna()
      base_teste = pd.read_csv(self.testPath)
      base_teste = base_teste.dropna()
      self.real_congestion_test = base_teste.iloc[:, 5:6].values
      frames = [self.base, self.base_teste]
      base_completa = pd.concat(frames)
      base_completa = base_completa.drop('#Ack', axis =1)
      base_completa = base_completa.drop('cwnd (Bytes/1000)', axis =1)
      base_completa = base_completa.drop('Network Situation', axis =1)
      base_completa = base_completa.drop('AckArrival(ms)', axis =1)
      base_completa = base_completa.drop('TSInsideAck(ms)', axis =1)
      base_completa = base_completa.drop('RTTAck(ms)', axis =1)
      entradas = base_completa[len(base_completa) - len(base_teste) - self.T:].values
      #base_teste_features = base_teste.iloc[:, [1,2,3,6]].values
      #print("#############",len(base_teste))
      self.len_base_teste = len(base_teste)
      X_teste = []

      for i in range(self.T, len(self.base_teste)+self.T): # para as duzentas previsoes, o mesmo tramanho do Teste.csv, ou seja 290-90
        X_teste.append(entradas[i-self.T:i,0:4])


      test_vectors = np.array(X_teste) # equivalente ao X_teste
      return test_vectors
    '''   
    def RefreshConfusionClientMatrix(self):
      #confrontar resultados
      self.confusionMatrizModelClient = np.full((2,2),random.randint(0,9))

    def RefreshConfusionServerMatrix(self):
      #confrontar resultados
      self.confusionMatrizModelServer = np.full((2,2),random.randint(0,9))
    '''       
    def ReceiveModelFromServer(self, parCandidateMatrix):
      print("Cliente ", self.id, " Recebido Modelo do Servidor")
      self.weightsServerModel.clear()
      for e in parCandidateMatrix :
        self.weightsServerModel.append(e)
      
    def SendModelToServer(self):
      self.centralServ.ReceiveModelsFromClients(self.id)
      print("Client", self.id, "Sending Model To Server")
    '''
    def TreinarModelo(self):
      print("Cliente ", self.id, "treinando Modelo")
      time.sleep(random.randint(0,9))
      self.RefreshConfusionClientMatrix()

    '''
    def GetPrevision(self): #evalueta indica que é uma avaliação do modelo recebido como parametro, no caso do servidor
      test_vectors = self.LoadTestData()
      previsoes = self.regressor.predict(test_vectors)
      #previsoes = parNeuralModel.predict(self.test_vectors)
     
      '''
      Observe que os previsores teste tem 90 quádruplas que conduzem ao resultado
      do último estado, da quádrupla 90. Prontamente preparado para pevisões....
      '''
      #classe_teste2 = np.array([])
      #previsoes2 =  np.array([])
      classe_teste2,previsoes2 = self.GetMapedMatrix(previsoes)
      '''
      for i in range(0, len(self.base_teste)):
        if(self.real_congestion_test[i] < 0.3):
          classe_teste2 = np.append(classe_teste2,0)
        elif (self.real_congestion_test[i,0] >= 0.3 and self.real_congestion_test[i] < 0.75):
          classe_teste2= np.append(classe_teste2,1)
        else:
          classe_teste2 = np.append(classe_teste2,2)

      for i in range(0, len(previsoes)):
        if(previsoes[i] < 0.3):
          previsoes2= np.append(previsoes2,0)
        elif (previsoes[i] >= 0.3 and previsoes[i] < 0.75):
          previsoes2= np.append(previsoes2,1)
        else:
          previsoes2 = np.append(previsoes2,2)
      '''
      self.currentConfusionMatriz = confusion_matrix(classe_teste2,previsoes2)
      return self.currentConfusionMatriz # Com a configuração corrente, essa é a matriz....
        

    def ServerModelIsBetter(self):
        regressorServer = Sequential()
        regressorServer.add(LSTM(units = 100, return_sequences = True, input_shape = (self.input_shape, 4)))# 4, pois são 4 previsores
        regressorServer.add(Dropout(0.3)) #zerar 30% das entradas para evitar o overfiting
        regressorServer.add(LSTM(units = 50, return_sequences = True))
        regressorServer.add(Dropout(0.3))
        regressorServer.add(LSTM(units = 50, return_sequences = True))
        regressorServer.add(Dropout(0.3))
        regressorServer.add(LSTM(units = 50))
        regressorServer.add(Dropout(0.3))

        '''
        Segundo https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
        as saídas de cada unidade da LSTM e, portanto, a saída global é a dimenção do número de previsores, que, no nosso
        caso, é 4. Daí esses 4 estão sendo levados em um softmax de tres neurônios, pois há tres categorias no final"
        ''' 
        regressorServer.add(Dense(units = 1, activation = 'sigmoid',name="client_eval_"+str(self.id)))
        regressorServer.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
        regressorServer.set_weights(self.weightsServerModel)
        return self.EvalueteServerModel(regressorServer)




class Server_FederatedAMA():
  '''
  During the aggregation,
  we first sum and average the neuron weights of each model. Then we store the
  average value in the designated global model position. In the neural network
  model, the same process is calculated for every neuron. After completing Al-
  gorithm 2, the central server aggregated model is sent to all the clients for the
  next learning phase
  '''
  def __init__(self):
    self.clients = []
    self.ReceivedModel = [False,False,False]
    self.consolidateWeightMatrix = []

  def PrintRegistredClients(self):
    print("Registred Clients")
    for x in self.clients:
      print("Client ", x.id)
  
  def RegisterClient(self, parCliente):
    self.clients.append(parCliente)
    #print("Cliente ", parCliente.id, " Regitrado com sucesso")

  '''
  def ReceiveModelsFromClient(self, parId):
   print("Recebido Modelos dos Cliente ", parId)    
   self.ConsolideModels()

  '''  
    

  def ConsolidateModels(self):

    '''
      #########################################################################################################################################
      listaBase = [["amarelo","verde","preto"], ["azul","abóbora","marrom"], ["uva","laranja","branco"]]
      arrays_in_Layer=[]
      lstTemp = []
      lst_arrays_in_Layer = []
      for layer in listaBase:
          for arr in layer:
              lstTemp.append(arr)
          arrays_in_Layer = [i for i in lstTemp]
          lst_arrays_in_Layer.append(arrays_in_Layer)
          lstTemp.clear()
      print (lst_arrays_in_Layer)
      
      
      #uma inspiraçao
      l=[]
      x=np.array() #weights
      y=np.array() #array of biases
      l.append(x)
      l.append(y)
      loaded_model.layers[0].set_weights(l) #loaded_model.layer[0] being the layer
  
    '''
    #client_layers = []

    numberClients = len(self.clients)
    #lstTemp = []
    #arrays_in_Layer=[]
    #lst_arrays_in_Layer = []
    #lst_consolidated_arrays_in_layer = [] #as camadas, composta por arraysconsolidados
    consolidated_arrays_in_layer = [] #arrays da camada
    #consolidated_array = None
    consolidated_model = []
    clients_weighted_models = []
    '''
    #Cada weightsClientModel é uma lista de numpyarrays crua "flatem", isto é
    se um modelo tem, por exemplo, 3 camadas e cada uma com dois arrays, o 
    weightsClientModel será uma lista de 6 numpy´s, com os dois primeiros vetores da 
    primeira camada no início, seguidos dos dois da segunta e, depois, os dois da terceira.
    Nada de listas. É uma Listona de numpys
    '''
    for client in self.clients:
        #print(client.weightsClientModel)
        #input("pesos acima")
        clients_weighted_models.append(client.weightsClientModel)
    
    
    #Criando a estrutura do Modelo
    #for model in client_weighted_model:
    '''
    count_arrays=0
    count_layers=0
    for arr in clients_weighted_models[0]:
       if(arr.shape[0]):
          count_arrays+=1
          consolidated_arrays_in_layer.append(np.full((2,2), 0))
    input(f"Quantidade de Arrays {count_arrays} do modelo")
        
    #for i in range(0,len (clients_weighted_models)):
    print("Shapes do modelo Consolidado")
    for j in range (0, len(consolidated_model)):
        for k in range (0, len (consolidated_model[j])):
            #print (f"({i},j{j},k{k}")
            print("consolidated shape--> (", consolidated_model[j][k].shape,")")
            
    input("final das shape")
    
    ############################################
    
    ###############Verificando as Shapes
    
    for i in range(0,len (clients_weighted_models)-1):
        print("Verificando as shapes")
        #input("mais uma shape")
        for j in range (0, len(clients_weighted_models[i])):
            for k in range (0, len (clients_weighted_models[i][j])):
                
                if(clients_weighted_models[i][j][k].shape != clients_weighted_models[i+1][j][k].shape):
                        print("shapes incompativeis")
                        #exit(1)
                       
    
    print("camadas cliente 0:", len(clients_weighted_models[0][0]))
    
    print("camadas modelo consolidado: ", len(consolidated_model[0]))
    '''
    
    for i in range(0,len (clients_weighted_models)):
       for j in range (0, len(clients_weighted_models[i])):
            if(i==0):
                consolidated_model.append(clients_weighted_models[i][j])#alocando a lista com os pesos do primeiro modelo.
            else:
                consolidated_model[j] = consolidated_model[j] + clients_weighted_models[i][j]
            if (i == numberClients - 1):
                consolidated_model[j] = consolidated_model[j]/numberClients
                
    
    '''
    
    for model in client_weighted_model:
        count_client = count_client + 1    
        for layer in model:
            for arr in layer:
                if(count_client==0):
                    consolidated_array = arr
                else:
                    consolidated_array = consolidated_array + arr
                if (count_client == numberClients - 1):
                    consolidated_array = consolidated_array/numberClients
                consolidated_arrays_in_layer.append(consolidated_array)
            lst_consolidated_arrays_in_layer.append(consolidated_arrays_in_layer)
    

    #obtendo os numpy de cada layer
    count_client=-1
    for model in client_weighted_model:
        count_client = count_client + 1    
        for layer in model:
            for arr in layer:
                if(count_client==0):
                    consolidated_array = arr
                else:
                    consolidated_array = consolidated_array + arr
                if (count_client == numberClients - 1):
                    consolidated_array = consolidated_array/numberClients
                consolidated_arrays_in_layer.append(consolidated_array)
            lst_consolidated_arrays_in_layer.append(consolidated_arrays_in_layer)
            
    self.consolidateWeightedMatrix = lst_consolidated_arrays_in_layer
                
              
      arrays_in_Layer = [i for i in lstTemp]
      lst_arrays_in_Layer.append(arrays_in_Layer)
      lstTemp.clear()
      
    client_layers.append(lst_arrays_in_Layer)
    
    #consolidando os pesos
    cout_client=-1
    for client_layer in client_layers:
      cont_client = count_client+1  
      for array_list in client_layer:
        for array in array_list:
            if(count_client==0):
                consolidated_array = array
            else:
                consolidated_array = consolidated_array + array
                
        consolidated_arrays_in_layer.append(consolidated_array)
      
    
    for i in range (1, numberClients):
      self.consolidateWeightedMatrix = self.consolidateWeightedMatrix + self.clients[i].weightsClientModel
    self.consolidateWeightedMatrix = self.consolidateWeightedMatrix/numberClients
    print("Models were consolidated")
    
    '''
    #self.consolidateWeightMatrix = consolidated_model # cuidado com isso!
    self.consolidateWeightMatrix.clear()
    for e in consolidated_model:
        self.consolidateWeightMatrix.append(e)

    
    print("Amostra Peso Clientes:")
    for client in self.clients:
      print("============================")
      print(client.weightsClientModel[0][0][0:4])
    print("Amostra Peso Consolidado: ")
    print(self.consolidateWeightMatrix[0][0][0:4])
    

  def FeedBackConsolidatedModel(self):
    for x in self.clients:
      x.ReceiveModelFromServer(self.consolidateWeightMatrix)
    print("Feedbacks Enviados")

  




 

Separando Lista

In [ ]:
listaBase = [["amarelo","verde","preto"], ["azul","abóbora","marrom"], ["uva","laranja","branco"]]
arrays_in_Layer=[]
lstTemp = []
lst_arrays_in_Layer = []
for layer in listaBase:
  for arr in layer:
    lstTemp.append(arr)
  arrays_in_Layer = [i for i in lstTemp]
  lst_arrays_in_Layer.append(arrays_in_Layer)
  lstTemp.clear()
print (lst_arrays_in_Layer)

[['amarelo', 'verde', 'preto'], ['azul', 'abóbora', 'marrom'], ['uva', 'laranja', 'branco']]


In [15]:
'''
Server::ConsolidateModels----------------Server::FeedBackConsolidatedModel--------------Client::RefreshModel
'''

#numClients = 3; #Can be gotten from user

#lstClients = [];

objServe = Server_FederatedAMA()

#for i in range (numClients):
  #objClient = Client(objServe,i)
  #lstClients.append(objClient)

#for x in lstClients:
  #print("Cliente ", x.id, "Criado com Sucesso");


#for x in lstClients:
 #objServe.RegisterClient(x)

#objServe.PrintRegistredClients()

'''
for i in range (0,10):
  print("==================Rodada ",i" ========================")
  for x in lstClients:
    x.TreinarModelo
  for x in lstClients:
    x.SendModelToServer
  objServe.ConsolidateModels
  objServe.FeedBackConsolidatedModel()
  for x in lstClients:
    x.AtualizarModelo();
'''

objClient1 = Client(0,'/content/drive/MyDrive/Colab Notebooks/Exp_000002/training_client01_Fri Feb 24 10_09_37.csv',
                               '/content/drive/MyDrive/Colab Notebooks/Exp_000002/test_Fri Feb 24 09_39_49.csv', 90);

#objClient1.LoadTainingDataSet()
objClient1.RefreshModel(parInitial=True)


objClient2 = Client(1,'/content/drive/MyDrive/Colab Notebooks/Exp_000002/training_client02_Fri Feb 24 10_09_37.csv',
                               '/content/drive/MyDrive/Colab Notebooks/Exp_000002/test_Fri Feb 24 09_39_49.csv', 90);

#objClient2.LoadTainingDataSet()
objClient2.RefreshModel(parInitial=True)



objClient3 = Client(2,'/content/drive/MyDrive/Colab Notebooks/Exp_000002/training_client03_10_1_2_2to10_35_0_2_trainingFri Feb 24 10_09_37.csv',
                               '/content/drive/MyDrive/Colab Notebooks/Exp_000002/test_Fri Feb 24 09_39_49.csv', 90);

#objClient3.LoadTainingDataSet()
objClient3.RefreshModel(parInitial=True)


Epoch 1/100


KeyboardInterrupt: ignored

In [13]:
###################Testando#################################

#objClient1.LoadTestData()
objClient1.GetPrevision()
print (objClient1.currentConfusionMatriz)

#objClient2.LoadTestData()
objClient2.GetPrevision()
print (objClient2.currentConfusionMatriz)


#objClient3.LoadTestData()
objClient3.GetPrevision()
print (objClient3.currentConfusionMatriz)

#########################Registrando no Servidor Central#######################################

objServe.RegisterClient(objClient1)
objServe.RegisterClient(objClient2)
objServe.RegisterClient(objClient3)

#########################Consolidando os modelos#######################################

objServe.ConsolidateModels()
objServe.FeedBackConsolidatedModel()

#########################Testando o novo modelo#######################################
#########################Atualizando, conforme o caso#######################################

objClient1.RefreshModel()
print (objClient1.currentConfusionMatriz)
objClient2.RefreshModel()
print (objClient2.currentConfusionMatriz)
objClient3.RefreshModel()
print (objClient2.currentConfusionMatriz)

#########################Novas matrizes, após atualização ou não dos modelos#######################################

objClient1.GetPrevision()
print (objClient1.currentConfusionMatriz)
objClient2.GetPrevision()
print (objClient2.currentConfusionMatriz)
objClient3.GetPrevision()
print (objClient3.currentConfusionMatriz)



AttributeError: ignored